In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
pd.set_option('display.max_columns', None)


# Reading in and combining all of the game log data into a single DF

In [793]:
def call_game_logs():
    #returns single df containing game logs for the last 4 seasons
    
    #reading in each game_log df
    gl_2020_2021 = pd.read_csv('data/game_logs_2020_2021.csv', skipinitialspace=True)
    
    gl_2019_2020 = pd.read_csv('data/game_logs_2019_2020.csv', skipinitialspace=True)
    
    gl_2018_2019 = pd.read_csv('data/game_logs_2018_2019.csv', skipinitialspace=True)
    
    gl_2017_2018 = pd.read_csv('data/game_logs_2017_2018.csv', skipinitialspace=True)
    
    gl_2016_2017 = pd.read_csv('data/game_logs_2016_2017.csv', skipinitialspace=True)
    
    gl_2015_2016 = pd.read_csv('data/game_logs_2015_2016.csv', skipinitialspace=True)
    
    #indexing current years game logs because data not current in the betting DF
    gl_2020_2021 = gl_2020_2021.iloc[66:]
    
    
    
    #dropping the Unnamed: 0 random columns from each df
    gl_2020_2021.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2019_2020.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2018_2019.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2017_2018.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2016_2017.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2015_2016.drop('Unnamed: 0', axis = 1, inplace= True)
    #concatenate all 3 game log df together
    
    gl_full = pd.concat([gl_2020_2021, gl_2019_2020, gl_2018_2019, gl_2017_2018, gl_2016_2017, gl_2015_2016])
    
    #replace "Game Date" column with a new column called ['Date']
    gl_full['Date'] = gl_full.iloc[:,2:3]
    gl_full.drop(gl_full.iloc[:,2:3], inplace = True, axis =1)
    
    
    #converting 'Date column to datetime'
    gl_full['Date'] = pd.to_datetime(gl_full['Date'])
    
    #converting to string to do goupby later 
    gl_full['Date'] = gl_full['Date'].astype(str)
    
    #reseting the index to make unique
    gl_full.reset_index(inplace =True)
    
    
    #dropping index after reseting 
    gl_full.drop(['index'], inplace = True, axis =1)
    
    return gl_full
    
    
full = call_game_logs()

In [794]:
full

,Team,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date
0,LAC,LAC vs. ATL,W,240,119,42,85,49.4,17,38,44.7,18,20,90.0,8,36,44,26,4,7,8,16,9,2021-03-22
1,ATL,ATL @ LAC,L,240,110,43,95,45.3,12,34,35.3,12,16,75.0,15,33,48,21,3,3,7,18,-9,2021-03-22
2,IND,IND @ MIL,L,240,113,45,111,40.5,12,50,24.0,11,12,91.7,11,29,40,27,8,3,6,16,-27,2021-03-22
3,MIL,MIL vs. IND,W,240,140,53,94,56.4,24,39,61.5,10,15,66.7,8,47,55,35,3,5,16,16,27,2021-03-22
4,HOU,HOU vs. TOR,W,240,117,41,90,45.6,14,36,38.9,21,26,80.8,10,37,47,28,7,7,12,21,18,2021-03-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,ATL,ATL vs. DET,L,240,94,37,82,45.1,8,27,29.6,12,15,80.0,7,33,40,22,9,4,15,25,-12,2015-10-27
13222,CHI,CHI vs. CLE,W,240,97,37,87,42.5,7,19,36.8,16,23,69.6,7,40,47,13,6,10,13,22,2,2015-10-27
13223,CLE,CLE @ CHI,L,240,95,38,94,40.4,9,29,31.0,10,17,58.8,11,39,50,26,5,7,11,21,-2,2015-10-27
13224,NOP,NOP @ GSW,L,240,95,35,83,42.2,6,18,33.3,19,27,70.4,8,25,33,21,9,3,19,26,-16,2015-10-27


In [474]:
#grouped = round(full_game_logs.groupby(['Team'], sort=False).rolling(3, min_periods=1).mean(),2).reset_index()

In [475]:
#grouped.sort_values('level_1') 

In [413]:
#do not know if I need this column later...
#full_game_logs = full_game_logs.rename(columns={full_game_logs.columns[1]: 'Match_up'})

In [775]:
#This DataFrame is to referencce the point totals when combining with the betting DF
#full



# transforming game logs stats to moving averages

In [1010]:
#creating the game_log df to input into the moving average function
df = call_game_logs()


#reversing the df so that the most recent game with be at the last index so that rolling averages will be calculated with the previous game
df = df.iloc[::-1]
#reseting new index after reversing the df
df.reset_index(inplace = True)

#dropping the old index to clean up the df
df.drop('index', inplace = True, axis = 1)
df

,Team,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date
0,GSW,GSW vs. NOP,W,240,111,41,96,42.7,9,30,30.0,20,22,90.9,21,35,56,29,8,7,20,29,16,2015-10-27
1,NOP,NOP @ GSW,L,240,95,35,83,42.2,6,18,33.3,19,27,70.4,8,25,33,21,9,3,19,26,-16,2015-10-27
2,CLE,CLE @ CHI,L,240,95,38,94,40.4,9,29,31.0,10,17,58.8,11,39,50,26,5,7,11,21,-2,2015-10-27
3,CHI,CHI vs. CLE,W,240,97,37,87,42.5,7,19,36.8,16,23,69.6,7,40,47,13,6,10,13,22,2,2015-10-27
4,ATL,ATL vs. DET,L,240,94,37,82,45.1,8,27,29.6,12,15,80.0,7,33,40,22,9,4,15,25,-12,2015-10-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,HOU,HOU vs. TOR,W,240,117,41,90,45.6,14,36,38.9,21,26,80.8,10,37,47,28,7,7,12,21,18,2021-03-22
13222,MIL,MIL vs. IND,W,240,140,53,94,56.4,24,39,61.5,10,15,66.7,8,47,55,35,3,5,16,16,27,2021-03-22
13223,IND,IND @ MIL,L,240,113,45,111,40.5,12,50,24.0,11,12,91.7,11,29,40,27,8,3,6,16,-27,2021-03-22
13224,ATL,ATL @ LAC,L,240,110,43,95,45.3,12,34,35.3,12,16,75.0,15,33,48,21,3,3,7,18,-9,2021-03-22


In [768]:
# moving_avemoving_ave = round(df.groupby(['Team']).rolling(7, min_periods=1).mean().shift(periods =1),2).reset_index()

# moving_avemoving_ave 

# This function converts all the data for game logs into moving averages

In [1031]:

def move_average(num_game):
    
    '''
    returns a df with the moving averages replacing their actual games stats for each game
    
    '''
    columns = ['MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
       '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', '+/-']
    
    #grouping by 'Team' and creating a df with rolling averages for each column
    moving_ave = df.groupby(['Team'])[columns].transform(lambda x: round(x.rolling(7, min_periods = 7).mean().shift(periods=1), 2)).reset_index()

    
    #sorting by old index to make indexes = to old indexes so that games line up
    #sort_by_old_index = moving_ave.sort_values('level_1')
    #print(sort_by_old_index)
    
    #reseting index to now match with old index and dropping meaningless columns
    #reset_to_old_index = sort_by_old_index.reset_index()
    #with_moving_average = reset_to_old_index.drop(['index', 'level_1'], axis = 1)
    
    
    #Adding the team columns back to the DF with the correct join
    moving_ave['Team'] = df['Team']
    
    
    #reversing back to the original order
    with_moving_average = moving_ave.iloc[::-1]
    #reseting new index after reversing the df
    with_moving_average.reset_index(inplace = True)

    #dropping the old index to clean up the df
    with_moving_average.drop(['index', 'level_0'], inplace = True, axis = 1)
        
   
    
    
    return with_moving_average

In [1030]:
columns = ['MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
       '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', '+/-']

moving_ave = df.groupby(['Team'])[columns].transform(lambda x: round(x.rolling(7, min_periods = 7).mean().shift(periods=1), 2)).reset_index()

In [1032]:

moving_ave

,index,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,13221,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71
13222,13222,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43
13223,13223,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29
13224,13224,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00


# Creating 3 different DF with moveing averages of 3, 5, and 7 games-

In [1033]:
##creating a df with moving averages of the last 3 games
#game_logs_average_3 = move_average(3)

##creating a df with moving averages of the last  games
#game_logs_average_5 = move_average(5)

##creating a df with moving averages of the last 7 games
game_logs_average_7 = move_average(7)

## creating a df with a moving average of the last 10 games
# game_logs_average_10 = move_average(10)

game_logs_average_7

/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team
0,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,LAC
1,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00,ATL
2,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,IND
3,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43,MIL
4,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,HOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL
13222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHI
13223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE
13224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOP


In [1034]:
# #putting team_1 as the first column
# game_logs_average_5 = game_logs_average_3[['Team','MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA',
#        'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-',
#        ]] 

game_logs_average_7 = game_logs_average_7[['Team','MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA',
       'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-',
       ]] 



In [1035]:
game_logs_average_7

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86
1,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00
2,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29
3,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43
4,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13222,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13223,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13224,NOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# This function convertsthe int64 object to a datetime object to match with 
# game logs lf

In [888]:
#converting int64 to correct datetime object

def add_year(df, year1, year2):
    
    for idx, string in enumerate(df['Date']):
        
        #checking to see if the date is more than 3 in length because that means it is before january 1
        if len(string) > 3:
            df['Date'][idx] = year1 + '-' + string[:2] +'-' + string[2:]
        else:
            
            #For some reason it turned january 10 into a 0 so i had to
            #add in a 1 before the 0 to make it a 10
            if string[-1] == '0':
                df['Date'][idx]= year2 + '-' + string[:2] + '-' + '1' + string[-1]
            
            #if the len of the string is 3 that means it is january first or after so
            #i added 2018 before the month and '-' in between the month and day.
            else:
                df['Date'][idx] = year2 + '-' +string[:1] + '-'  + string[2:]
            
        
            
    return df['Date']




# Reading in and combing all of the betting information for last 4 seasons

In [800]:
def read_in_bet():
    
    #returns a full betting df of the regular season games for the last four seasons
    
    # reading in and indexing 2017_18 index these df here[:2460] to exclude the playoff betting information
    bet_2015_16 = pd.read_csv('data/nba odds 2015-16.csv')
    bet_2015_16 = bet_2015_16[:2460]
    #Indexing to exclude the last columns 
    #we do not want
    #bet_2017_18 = bet_2017_18.iloc[:,:13]
    
    #calling add year function to change the datetime object to match the gamelogs for 2018-2019 & converting 
    #'Date' to a string object
    bet_2015_16['Date'] = bet_2015_16['Date'].astype(str)
    bet_2015_16['Date'] = add_year(bet_2015_16, '2017', '2018')
    
    
    # reading in and indexing 2017_18 index these df here[:2460] to exclude the playoff betting information
    bet_2016_17 = pd.read_csv('data/nba_odds_2016-17.csv')
    bet_2016_17 = bet_2016_17[:2460]
    #Indexing to exclude the last columns 
    #we do not want
    #bet_2017_18 = bet_2017_18.iloc[:,:13]
    
    #calling add year function to change the datetime object to match the gamelogs for 2018-2019 & converting 
    #'Date' to a string object
    bet_2016_17['Date'] = bet_2016_17['Date'].astype(str)
    bet_2016_17['Date'] = add_year(bet_2016_17, '2017', '2018')
    
    
    
    
    
    # reading in and indexing 2017_18 index these df here[:2460] to exclude the playoff betting information
    bet_2017_18 = pd.read_csv('data/nba_odds_2017-18.csv')
    bet_2017_18 = bet_2017_18[:2460]
    #Indexing to exclude the last columns 
    #we do not want
    bet_2017_18 = bet_2017_18.iloc[:,:13]
    
    #calling add year function to change the datetime object to match the gamelogs for 2018-2019 & converting 
    #'Date' to a string object
    bet_2017_18['Date'] = bet_2017_18['Date'].astype(str)
    bet_2017_18['Date'] = add_year(bet_2017_18, '2017', '2018')
    
    
    
    # reading in and indexing 2018_19 index these df here[:2460] to exclude the playoff betting information
    bet_2018_19 = pd.read_csv('data/nba_odds_2018_19.csv')
    bet_2018_19 = bet_2018_19[:2460]
    
    #calling add year function to change the datetime object to match the gamelogs for 2018-2019 & converting 
    #'Date' to a string object
    bet_2018_19['Date'] = bet_2018_19['Date'].astype(str)
    bet_2018_19['Date'] = add_year(bet_2018_19, '2018', '2019')

    
    
    #reading in and indexing 2019_2020 to exclude playoffs **covid season
    #This include covid year so index [:2118] to only include the regular season
    bet_2019_2020 = pd.read_csv('data/nba_odds_2019_20.csv')
    bet_2019_2020 = bet_2019_2020[:2118]
    
    #calling add year function to change the datetime object to match the gamelogs for 2019-2020 & converting 
    #'Date' to a string object
    bet_2019_2020['Date'] = bet_2019_2020['Date'].astype(str)
    bet_2019_2020['Date'] = add_year(bet_2019_2020, '2019', '2020')
    
    
    
    #reading in and indexing 2019_2020 to exclude playoffs **covid season
    bet_2020_2021 = pd.read_csv('data/nba_odds_2020-21.csv')
    #not indexing in the betting data because it is the current year and does not include the playoffs
    bet_2020_2021 = bet_2020_2021
    
    #calling add year function to change the datetime object to match the gamelogs for 2019-2020 & converting 
    #'Date' to a string object
    bet_2020_2021['Date'] = bet_2020_2021['Date'].astype(str)
    bet_2020_2021['Date'] = add_year(bet_2020_2021, '2020', '2022')
    
    #just need to include the betting informatin for the current season
    ##### Finish when ready to run the model
    
    #creating a single DF for all of the seasons of betting information Indexing to exclude the last columns 
    #we do not want
    bet_full = pd.concat([bet_2020_2021, bet_2019_2020, bet_2018_19, bet_2017_18, bet_2016_17, bet_2015_16])
    
    #reseting new index after reversing the df
    bet_full.reset_index(inplace = True)

    #dropping the old index to clean up the df
    bet_full.drop('index', inplace = True, axis = 1)
    
    
    return bet_full
full_odds = read_in_bet()

/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipyke

In [801]:
full_odds

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,2020-12-22,501.0,V,GoldenState,25.0,20.0,26.0,28.0,99.0,228,234.5,265.0,1
1,2020-12-22,502.0,H,Brooklyn,40.0,23.0,36.0,26.0,125.0,5.5,7.5,-320.0,115
2,2020-12-22,503.0,V,LAClippers,39.0,17.0,33.0,27.0,116.0,223,219,115.0,108.5
3,2020-12-22,504.0,H,LALakers,19.0,35.0,24.0,31.0,109.0,2.5,2,-135.0,3.5
4,2020-12-23,551.0,V,Charlotte,23.0,21.0,36.0,34.0,114.0,215,3,-140.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,2017-41-3.0,724.0,H,GoldenState,37.0,33.0,32.0,23.0,125.0,16,17,-9900.0,4.5
13222,2017-41-3.0,725.0,V,Denver,31.0,25.0,23.0,20.0,99.0,214,218,330.0,109.5
13223,2017-41-3.0,726.0,H,Portland,27.0,31.0,29.0,20.0,107.0,9.5,9.5,-400.0,5
13224,2017-41-3.0,727.0,V,LAClippers,27.0,28.0,20.0,30.0,105.0,205,206,165.0,103


# Function to replace all the team string values to their acronym to combine with game log DF

In [802]:

def replace_string():
    
    
    full_odds['Team'] = full_odds['Team'].str.replace('GoldenState','GSW')
    full_odds['Team'] = full_odds['Team'].str.replace('Brooklyn', 'BKN')
    full_odds['Team'] = full_odds['Team'].str.replace('LAClippers', 'LAC')
    full_odds['Team'] = full_odds['Team'].str.replace('Charlotte', 'CHA')
    full_odds['Team'] = full_odds['Team'].str.replace('LALakers', 'LAL')
    full_odds['Team'] = full_odds['Team'].str.replace('Cleveland', 'CLE')
    full_odds['Team'] = full_odds['Team'].str.replace('NewYork','NYK')
    full_odds['Team'] = full_odds['Team'].str.replace('Indiana', 'IND')
    full_odds['Team'] = full_odds['Team'].str.replace('Miami', 'MIA')
    full_odds['Team'] = full_odds['Team'].str.replace('Orlando', 'ORL')
    full_odds['Team'] = full_odds['Team'].str.replace('Washington', 'WAS')
    full_odds['Team'] = full_odds['Team'].str.replace('Philadelphia', 'PHI')
    full_odds['Team'] = full_odds['Team'].str.replace('Milwaukee', 'MIL')
    full_odds['Team'] = full_odds['Team'].str.replace('Boston', 'BOS')
    full_odds['Team'] = full_odds['Team'].str.replace('NewOrleans', 'NOP')
    full_odds['Team'] = full_odds['Team'].str.replace('Toronto', 'TOR')
    full_odds['Team'] = full_odds['Team'].str.replace('Atlanta', 'ATL')
    full_odds['Team'] = full_odds['Team'].str.replace('Chicago', 'CHI')
    full_odds['Team'] = full_odds['Team'].str.replace('SanAntonio', 'SAS')
    full_odds['Team'] = full_odds['Team'].str.replace('Memphis', 'MEM')
    full_odds['Team'] = full_odds['Team'].str.replace('Detroit', 'DET')
    full_odds['Team'] = full_odds['Team'].str.replace('Minnesota', 'MIN')
    full_odds['Team'] = full_odds['Team'].str.replace('Sacramento', 'SAC')
    full_odds['Team'] = full_odds['Team'].str.replace('Denver', 'DEN')
    full_odds['Team'] = full_odds['Team'].str.replace('Utah', 'UTA')
    full_odds['Team'] = full_odds['Team'].str.replace('Portland', 'POR')
    full_odds['Team'] = full_odds['Team'].str.replace('Dallas', 'DAL')
    full_odds['Team'] = full_odds['Team'].str.replace('Phoenix', 'PHX')
    full_odds['Team'] = full_odds['Team'].str.replace('OklahomaCity', 'OKC')
    full_odds['Team'] = full_odds['Team'].str.replace('Houston', 'HOU')
    
    #converting Date in full odds to string to convert to datetime later 
    
    full_odds['Date']   =  full_odds['Date'].astype(str) 
    
    return full_odds
    
full_odd_real = replace_string()

In [803]:
full_odd_real

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,2020-12-22,501.0,V,GSW,25.0,20.0,26.0,28.0,99.0,228,234.5,265.0,1
1,2020-12-22,502.0,H,BKN,40.0,23.0,36.0,26.0,125.0,5.5,7.5,-320.0,115
2,2020-12-22,503.0,V,LAC,39.0,17.0,33.0,27.0,116.0,223,219,115.0,108.5
3,2020-12-22,504.0,H,LAL,19.0,35.0,24.0,31.0,109.0,2.5,2,-135.0,3.5
4,2020-12-23,551.0,V,CHA,23.0,21.0,36.0,34.0,114.0,215,3,-140.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13221,2017-41-3.0,724.0,H,GSW,37.0,33.0,32.0,23.0,125.0,16,17,-9900.0,4.5
13222,2017-41-3.0,725.0,V,DEN,31.0,25.0,23.0,20.0,99.0,214,218,330.0,109.5
13223,2017-41-3.0,726.0,H,POR,27.0,31.0,29.0,20.0,107.0,9.5,9.5,-400.0,5
13224,2017-41-3.0,727.0,V,LAC,27.0,28.0,20.0,30.0,105.0,205,206,165.0,103


In [718]:
# # a list of all 30 teams in the nba to change into the acronym to combine dataframes later

# 'GoldenState' = 'GSW'
# 'Brooklyn' = 'BKN'
# 'LAClippers' = 'LAC'
# 'Charlotte' = 'CHA'
# 'LALakers' = 'LAL'
# 'Cleveland' = 'CLE'
# 'NewYork' = 'NYK'
# 'Indiana' = 'IND'
# 'Miami'  = 'MIA'
# 'Orlando' = 'ORL'
# 'Washington' = 'WAS'
# 'Philadelphia' = 'PHI'
# 'Milwaukee' = 'MIL'
# 'Boston' = 'BOS'
# 'NewOrleans' = 'NOP'
# 'Toronto' = 'TOR'
# 'Atlanta' = 'ATL'
# 'Chicago' = 'CHI'
# 'SanAntonio' = 'SAS'
# 'Memphis' = 'MEM'
# 'Detroit' = 'DET'
# 'Minnesota' = 'MIN'
# 'Sacramento' = 'SAC'
# 'Denver' = 'DEN'
# 'Utah' = 'UTA'
# 'Portland' = 'POR'
# 'Dallas' = 'DAL'
# 'Phoenix' = 'PHX'
# 'OklahomaCity' = 'OKC'
# 'Houston' = 'HOU'



In [719]:
# https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba/nbaoddsarchives.htm

# This function is for combing the gome log df with out moving averages

- ## Merging every other row in game log DF so each row represents single game

In [806]:
def combine_game_logs():
    
    #indexing every row and using concat to add to the row above
    merge_logs = pd.concat((full.iloc[::2].reset_index(drop=True), 
                    full.iloc[1::2].reset_index(drop=True)),
                   axis=1)
    
    
    #creating two different df to rename the second half of the columns
    old = merge_logs.iloc[:, :24]
    new = merge_logs.iloc[:, 24:]
    #adding suffix to column names in 2nd split DF
    new.columns = [str(col) + '_2' for col in new.columns]
    
    
    #re-merging the two DFs after adding a suffix to the second's column names
    merge_logs_final = pd.concat((old.reset_index(drop=True), 
                    new.reset_index(drop=True)),
                   axis=1)
    
    
    #converting the date column to a string value to merge with bet DF later
    merge_logs_final['Date'] = merge_logs_final['Date'].astype(str)
    
    
    #renamong [matchup column because it I couldnt access it previously]
    merge_logs_final = merge_logs_final.rename(columns={merge_logs_final.columns[1]: 'Match_up'})
    
    
    merge_logs_final['Total_pts'] = merge_logs_final['PTS'] + merge_logs_final['PTS_2']
    
    
    #creating a new column which is our y value that is total points for the game.
    return merge_logs_final

total_points_game_log =  combine_game_logs()

In [917]:
total_points_game_log

,Team,Match_up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date,Team_2,Match Up_2,W/L_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Date_2,Total_pts,home,visitor
0,LAC,LAC vs. ATL,W,240,119,42,85,49.4,17,38,44.7,18,20,90.0,8,36,44,26,4,7,8,16,9,2021-03-22,ATL,ATL @ LAC,L,240,110,43,95,45.3,12,34,35.3,12,16,75.0,15,33,48,21,3,3,7,18,-9,2021-03-22,229,LAC,ATL
1,IND,IND @ MIL,L,240,113,45,111,40.5,12,50,24.0,11,12,91.7,11,29,40,27,8,3,6,16,-27,2021-03-22,MIL,MIL vs. IND,W,240,140,53,94,56.4,24,39,61.5,10,15,66.7,8,47,55,35,3,5,16,16,27,2021-03-22,253,MIL,IND
2,HOU,HOU vs. TOR,W,240,117,41,90,45.6,14,36,38.9,21,26,80.8,10,37,47,28,7,7,12,21,18,2021-03-22,TOR,TOR @ HOU,L,240,99,37,87,42.5,10,40,25.0,15,19,78.9,8,36,44,19,6,5,15,24,-18,2021-03-22,216,HOU,TOR
3,BOS,BOS @ MEM,L,265,126,47,90,52.2,17,35,48.6,15,24,62.5,5,38,43,31,3,6,13,18,-6,2021-03-22,MEM,MEM vs. BOS,W,265,132,55,110,50.0,12,35,34.3,10,15,66.7,17,37,54,38,8,6,10,20,6,2021-03-22,258,MEM,BOS
4,UTA,UTA @ CHI,W,240,120,45,92,48.9,17,39,43.6,13,15,86.7,10,36,46,25,9,10,9,16,25,2021-03-22,CHI,CHI vs. UTA,L,240,95,39,96,40.6,6,26,23.1,11,12,91.7,14,35,49,21,5,3,14,14,-25,2021-03-22,215,CHI,UTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,PHX,PHX vs. DAL,L,240,95,34,87,39.1,6,24,25.0,21,33,63.6,13,40,53,15,3,3,18,30,-16,2015-10-28,DAL,DAL @ PHX,W,240,111,40,85,47.1,10,21,47.6,21,31,67.7,6,42,48,24,7,3,8,25,16,2015-10-28,206,PHX,DAL
6609,LAL,LAL vs. MIN,L,240,111,35,90,38.9,9,34,26.5,32,35,91.4,12,36,48,18,2,4,14,29,-1,2015-10-28,MIN,MIN @ LAL,W,240,112,38,83,45.8,5,20,25.0,31,37,83.8,6,33,39,24,3,4,8,26,1,2015-10-28,223,LAL,MIN
6610,DET,DET @ ATL,W,240,106,37,96,38.5,12,29,41.4,20,26,76.9,23,36,59,23,5,3,15,15,12,2015-10-27,ATL,ATL vs. DET,L,240,94,37,82,45.1,8,27,29.6,12,15,80.0,7,33,40,22,9,4,15,25,-12,2015-10-27,200,ATL,DET
6611,CHI,CHI vs. CLE,W,240,97,37,87,42.5,7,19,36.8,16,23,69.6,7,40,47,13,6,10,13,22,2,2015-10-27,CLE,CLE @ CHI,L,240,95,38,94,40.4,9,29,31.0,10,17,58.8,11,39,50,26,5,7,11,21,-2,2015-10-27,192,CHI,CLE


# This function is for combing the gome log df with moving averages

In [1036]:
# game_logs_average_3    ----> represents moving averages of the last 3 games
# game_logs_average_5    ----> represents moving averages of the last 5 games
# game_logs_average_7    ----> represents moving averages of the last 7 games

def combine_game_logs():
    
    #indexing every row and using concat to add to the row above
    merge_logs = pd.concat((game_logs_average_7.iloc[::2].reset_index(drop=True), 
                    game_logs_average_7.iloc[1::2].reset_index(drop=True)),
                   axis=1)
    
    
    #creating two different df to rename the second half of the columns
    old = merge_logs.iloc[:, :21]
    new = merge_logs.iloc[:, 21:]
    #adding suffix to column names in 2nd split DF
    new.columns = [str(col) + '_2' for col in new.columns]
    
    
    #re-merging the two DFs after adding a suffix to the second's column names
    merge_logs_final = pd.concat((old.reset_index(drop=True), 
                    new.reset_index(drop=True)),
                   axis=1)
    
    
    #converting the date column to a string value to merge with bet DF later
    #merge_logs_final['Date'] = merge_logs_final['Date'].astype(str)
    
    
    #renamong [matchup column because it I couldnt access it previously]
    #merge_logs_final = merge_logs_final.rename(columns={merge_logs_final.columns[1]: 'Match_up'})
    
    
    #merge_logs_final['Total_pts'] = merge_logs_final['PTS'] + merge_logs_final['PTS_2']
    
    
    #creating a new column which is our y value that is total points for the game.
    return merge_logs_final

final_game_log =  combine_game_logs()

In [1037]:
#This DF has the moving averages for whateer period is specified
final_game_log


#This DF has all the game day so we can combine it with the moving averages to get the correct total points or the target variable
#total_points_game_log

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00
1,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43
2,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,TOR,240.00,110.43,38.86,91.00,42.81,16.14,44.00,36.79,16.57,20.86,79.86,9.00,28.29,37.29,26.29,8.29,4.43,12.86,24.00,-8.43
3,BOS,240.00,114.57,43.43,88.86,49.17,14.14,37.00,38.17,13.57,18.86,76.07,11.43,32.57,44.00,24.57,8.00,5.86,13.29,20.14,1.71,MEM,240.00,107.57,40.14,89.86,44.56,8.86,28.00,32.03,18.43,24.14,78.74,10.43,37.00,47.43,24.43,8.14,4.14,13.43,19.29,-2.29
4,UTA,243.57,119.14,40.86,89.29,45.84,16.71,42.71,38.99,20.71,26.29,78.73,11.00,35.57,46.57,22.71,7.71,5.43,15.14,20.00,-1.14,CHI,243.57,108.86,41.29,89.57,46.14,13.57,34.71,40.04,12.71,15.71,83.87,11.14,36.71,47.86,27.43,6.00,4.71,16.43,19.43,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6609,LAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6610,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6611,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1038]:
#re_creating the total points column because it the team points column is not the average points, and we need total points to merge with game odds df

final_game_log['Total_pts'] = total_points_game_log['Total_pts']

In [1039]:
final_game_log

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Total_pts
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00,229
1,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43,253
2,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,TOR,240.00,110.43,38.86,91.00,42.81,16.14,44.00,36.79,16.57,20.86,79.86,9.00,28.29,37.29,26.29,8.29,4.43,12.86,24.00,-8.43,216
3,BOS,240.00,114.57,43.43,88.86,49.17,14.14,37.00,38.17,13.57,18.86,76.07,11.43,32.57,44.00,24.57,8.00,5.86,13.29,20.14,1.71,MEM,240.00,107.57,40.14,89.86,44.56,8.86,28.00,32.03,18.43,24.14,78.74,10.43,37.00,47.43,24.43,8.14,4.14,13.43,19.29,-2.29,258
4,UTA,243.57,119.14,40.86,89.29,45.84,16.71,42.71,38.99,20.71,26.29,78.73,11.00,35.57,46.57,22.71,7.71,5.43,15.14,20.00,-1.14,CHI,243.57,108.86,41.29,89.57,46.14,13.57,34.71,40.04,12.71,15.71,83.87,11.14,36.71,47.86,27.43,6.00,4.71,16.43,19.43,2.00,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206
6609,LAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223
6610,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200
6611,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192


In [907]:
def combine_game_odd():
    
    #indexing every row and using concat to add to the row above
    merge_odds = pd.concat((full_odd_real.iloc[::2].reset_index(drop=True), 
                    full_odd_real.iloc[1::2].reset_index(drop=True)),
                   axis=1)
    
    
    #creating two different df to rename the second half of the columns
    old = merge_odds.iloc[:, :13]
    new = merge_odds.iloc[:, 13:]
    #adding suffix to column names in 2nd split DF
    new.columns = [str(col) + '_2' for col in new.columns]
    
    
    #re-merging the two DFs after adding a suffix to the second's column names
    merge_odds_final = pd.concat((old.reset_index(drop=True), 
                    new.reset_index(drop=True)),
                   axis=1)
    
    
    
    return merge_odds_final



final_odds = combine_game_odd()

In [908]:
final_odds['Total_pts'] = final_odds['Final'] + final_odds['Final_2']


In [909]:
final_odds

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,Total_pts
0,2020-12-22,501.0,V,GSW,25.0,20.0,26.0,28.0,99.0,228,234.5,265.0,1,2020-12-22,502.0,H,BKN,40.0,23.0,36.0,26.0,125.0,5.5,7.5,-320.0,115,224.0
1,2020-12-22,503.0,V,LAC,39.0,17.0,33.0,27.0,116.0,223,219,115.0,108.5,2020-12-22,504.0,H,LAL,19.0,35.0,24.0,31.0,109.0,2.5,2,-135.0,3.5,225.0
2,2020-12-23,551.0,V,CHA,23.0,21.0,36.0,34.0,114.0,215,3,-140.0,7,2020-12-23,552.0,H,CLE,24.0,41.0,28.0,28.0,121.0,1,217,120.0,107,235.0
3,2020-12-23,553.0,V,NYK,33.0,33.0,16.0,25.0,107.0,214.5,215.5,270.0,107.5,2020-12-23,554.0,H,IND,35.0,26.0,27.0,33.0,121.0,8.5,7.5,-330.0,7.5,228.0
4,2020-12-23,555.0,V,MIA,29.0,24.0,30.0,24.0,107.0,3.5,4.5,-190.0,5,2020-12-23,556.0,H,ORL,27.0,29.0,23.0,34.0,113.0,220.5,219,170.0,106.5,220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,2017-41-3.0,719.0,V,ORL,22.0,28.0,29.0,24.0,103.0,209,211.5,280.0,0.5,2017-41-3.0,720.0,H,CHA,38.0,28.0,20.0,31.0,117.0,10,7.5,-340.0,104,220.0
6609,2017-41-3.0,721.0,V,UTA,21.0,36.0,18.0,21.0,96.0,8,3.5,-155.0,100,2017-41-3.0,722.0,H,LAL,19.0,23.0,24.0,35.0,101.0,189.5,196,135.0,3,197.0
6610,2017-41-3.0,723.0,V,MEM,23.0,27.0,31.0,23.0,104.0,213.5,213,2000.0,105.5,2017-41-3.0,724.0,H,GSW,37.0,33.0,32.0,23.0,125.0,16,17,-9900.0,4.5,229.0
6611,2017-41-3.0,725.0,V,DEN,31.0,25.0,23.0,20.0,99.0,214,218,330.0,109.5,2017-41-3.0,726.0,H,POR,27.0,31.0,29.0,20.0,107.0,9.5,9.5,-400.0,5,206.0


In [910]:
#renaming columns to identify home team or away team
final_odds.rename(columns={'Team':'visitor'}, inplace=True)
final_odds.rename(columns={'Team_2':'home'}, inplace=True)

In [997]:
#creating new columns so that we can join bet DF with game_logs df 
total_points_game_log['home'] = [total_points_game_log['Team'][i] if '@' not in v else total_points_game_log['Team_2'][i] for i, v in enumerate(total_points_game_log['Match_up'])]
total_points_game_log['visitor'] = [total_points_game_log['Team'][i] if '@' in v else total_points_game_log['Team_2'][i] for i, v in enumerate(total_points_game_log['Match_up'])]

In [1040]:
# creating two new columns in the moving average DF because we lost the match_up column in the game logs after we did a groupby
final_game_log['home']  =  total_points_game_log['home']
final_game_log['visitor']  =  total_points_game_log['visitor']
#final_game_log['Total_points']

final_game_log

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Total_pts,home,visitor
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00,229,LAC,ATL
1,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43,253,MIL,IND
2,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,TOR,240.00,110.43,38.86,91.00,42.81,16.14,44.00,36.79,16.57,20.86,79.86,9.00,28.29,37.29,26.29,8.29,4.43,12.86,24.00,-8.43,216,HOU,TOR
3,BOS,240.00,114.57,43.43,88.86,49.17,14.14,37.00,38.17,13.57,18.86,76.07,11.43,32.57,44.00,24.57,8.00,5.86,13.29,20.14,1.71,MEM,240.00,107.57,40.14,89.86,44.56,8.86,28.00,32.03,18.43,24.14,78.74,10.43,37.00,47.43,24.43,8.14,4.14,13.43,19.29,-2.29,258,MEM,BOS
4,UTA,243.57,119.14,40.86,89.29,45.84,16.71,42.71,38.99,20.71,26.29,78.73,11.00,35.57,46.57,22.71,7.71,5.43,15.14,20.00,-1.14,CHI,243.57,108.86,41.29,89.57,46.14,13.57,34.71,40.04,12.71,15.71,83.87,11.14,36.71,47.86,27.43,6.00,4.71,16.43,19.43,2.00,215,CHI,UTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206,PHX,DAL
6609,LAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223,LAL,MIN
6610,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,ATL,DET
6611,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE


# Merging the averaged game logs with the odds DF on 'visitor', 'home', 'Total_pts'

In [1041]:

merged_odds_logs = final_game_log.merge(final_odds, how = 'left', on = ['visitor', 'home', 'Total_pts'])

In [1042]:
merged_odds_logs

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Total_pts,home,visitor,Date,Rot,VH,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Date_2,Rot_2,VH_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00,229,LAC,ATL,2022-3-2,545.0,NaN,29.0,34.0,27.0,20.0,110.0,226,225,220.0,110,2022-3-2,546.0,NaN,30.0,18.0,34.0,37.0,119.0,6.5,6.5,-260.0,7
1,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43,253,MIL,IND,2022-3-2,539.0,NaN,26.0,34.0,34.0,19.0,113.0,232,232.5,230.0,2.5,2022-3-2,540.0,NaN,48.0,35.0,27.0,30.0,140.0,10,6.5,-270.0,114
2,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,TOR,240.00,110.43,38.86,91.00,42.81,16.14,44.00,36.79,16.57,20.86,79.86,9.00,28.29,37.29,26.29,8.29,4.43,12.86,24.00,-8.43,216,HOU,TOR,2022-3-2,541.0,NaN,28.0,32.0,26.0,13.0,99.0,8.5,8.5,-380.0,8.5,2022-3-2,542.0,NaN,27.0,38.0,23.0,29.0,117.0,223.5,225.5,320.0,111
3,BOS,240.00,114.57,43.43,88.86,49.17,14.14,37.00,38.17,13.57,18.86,76.07,11.43,32.57,44.00,24.57,8.00,5.86,13.29,20.14,1.71,MEM,240.00,107.57,40.14,89.86,44.56,8.86,28.00,32.03,18.43,24.14,78.74,10.43,37.00,47.43,24.43,8.14,4.14,13.43,19.29,-2.29,258,MEM,BOS,2022-3-2,537.0,NaN,30.0,35.0,21.0,31.0,126.0,221.5,218.5,165.0,113.5,2022-3-2,538.0,NaN,24.0,36.0,34.0,23.0,132.0,1,4.5,-185.0,4
4,UTA,243.57,119.14,40.86,89.29,45.84,16.71,42.71,38.99,20.71,26.29,78.73,11.00,35.57,46.57,22.71,7.71,5.43,15.14,20.00,-1.14,CHI,243.57,108.86,41.29,89.57,46.14,13.57,34.71,40.04,12.71,15.71,83.87,11.14,36.71,47.86,27.43,6.00,4.71,16.43,19.43,2.00,215,CHI,UTA,2022-3-2,543.0,NaN,29.0,27.0,33.0,31.0,120.0,8.5,9.5,-500.0,1,2022-3-2,544.0,NaN,23.0,19.0,29.0,24.0,95.0,231,227.5,400.0,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7276,LAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223,LAL,MIN,2017-10-28.0,727.0,V,22.0,29.0,37.0,24.0,112.0,202.5,202.5,120.0,1.5,2017-10-28.0,728.0,H,31.0,29.0,35.0,16.0,111.0,3.5,2.5,-140.0,104.5
7277,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,ATL,DET,2017-10-27.0,503.0,V,25.0,23.0,34.0,24.0,106.0,197.5,196.5,255.0,98,2017-10-27.0,504.0,H,25.0,18.0,23.0,28.0,94.0,7,7,-320.0,7
7278,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE,2017-33-0.0,703.0,V,28.0,22.0,21.0,22.0,93.0,5.5,5.5,-220.0,0.5,2017-33-0.0,704.0,H,22.0,19.0,37.0,21.0,99.0,210.5,212,190.0,102.5
7279,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE,2017-10-27.0,501.0,V,17.0,23.0,28.0,27.0,95.0,197.5,198.5,160.0,97,2017-10-27.0,502.0,H,26.0,20.0,25.0,26.0,97.0,2,4,-180.0,pk


In [1043]:
#creating a 'over under' column to isolate the game totals
merged_odds_logs['over_under'] = [merged_odds_logs['Open'][i] if len(str(v)) > 2 else merged_odds_logs['Open_2'][i] for i, v in enumerate(merged_odds_logs['Open'])]

In [1044]:
#creating a total column because some of the information was from point spread so had to make sure it was an over under total
merged_odds_logs['totals'] = [merged_odds_logs['Open_2'][i] if len(str(v)) == 3 and '.' in str(v) else merged_odds_logs['over_under'][i] for i, v in enumerate(merged_odds_logs['over_under'])]
merged_odds_logs
# #renaming Match Up_2 column because for some reason it could not be accessed
# logs_odds = logs_odds.rename(columns={logs_odds.columns[25]: 'Match_up_2'})
# logs_odds = logs_odds.rename(columns={logs_odds.columns[25]: 'Match_up_2'})

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Total_pts,home,visitor,Date,Rot,VH,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Date_2,Rot_2,VH_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,over_under,totals
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00,229,LAC,ATL,2022-3-2,545.0,NaN,29.0,34.0,27.0,20.0,110.0,226,225,220.0,110,2022-3-2,546.0,NaN,30.0,18.0,34.0,37.0,119.0,6.5,6.5,-260.0,7,226,226
1,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43,253,MIL,IND,2022-3-2,539.0,NaN,26.0,34.0,34.0,19.0,113.0,232,232.5,230.0,2.5,2022-3-2,540.0,NaN,48.0,35.0,27.0,30.0,140.0,10,6.5,-270.0,114,232,232
2,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,TOR,240.00,110.43,38.86,91.00,42.81,16.14,44.00,36.79,16.57,20.86,79.86,9.00,28.29,37.29,26.29,8.29,4.43,12.86,24.00,-8.43,216,HOU,TOR,2022-3-2,541.0,NaN,28.0,32.0,26.0,13.0,99.0,8.5,8.5,-380.0,8.5,2022-3-2,542.0,NaN,27.0,38.0,23.0,29.0,117.0,223.5,225.5,320.0,111,8.5,223.5
3,BOS,240.00,114.57,43.43,88.86,49.17,14.14,37.00,38.17,13.57,18.86,76.07,11.43,32.57,44.00,24.57,8.00,5.86,13.29,20.14,1.71,MEM,240.00,107.57,40.14,89.86,44.56,8.86,28.00,32.03,18.43,24.14,78.74,10.43,37.00,47.43,24.43,8.14,4.14,13.43,19.29,-2.29,258,MEM,BOS,2022-3-2,537.0,NaN,30.0,35.0,21.0,31.0,126.0,221.5,218.5,165.0,113.5,2022-3-2,538.0,NaN,24.0,36.0,34.0,23.0,132.0,1,4.5,-185.0,4,221.5,221.5
4,UTA,243.57,119.14,40.86,89.29,45.84,16.71,42.71,38.99,20.71,26.29,78.73,11.00,35.57,46.57,22.71,7.71,5.43,15.14,20.00,-1.14,CHI,243.57,108.86,41.29,89.57,46.14,13.57,34.71,40.04,12.71,15.71,83.87,11.14,36.71,47.86,27.43,6.00,4.71,16.43,19.43,2.00,215,CHI,UTA,2022-3-2,543.0,NaN,29.0,27.0,33.0,31.0,120.0,8.5,9.5,-500.0,1,2022-3-2,544.0,NaN,23.0,19.0,29.0,24.0,95.0,231,227.5,400.0,116,8.5,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7276,LAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223,LAL,MIN,2017-10-28.0,727.0,V,22.0,29.0,37.0,24.0,112.0,202.5,202.5,120.0,1.5,2017-10-28.0,728.0,H,31.0,29.0,35.0,16.0,111.0,3.5,2.5,-140.0,104.5,202.5,202.5
7277,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,ATL,DET,2017-10-27.0,503.0,V,25.0,23.0,34.0,24.0,106.0,197.5,196.5,255.0,98,2017-10-27.0,504.0,H,25.0,18.0,23.0,28.0,94.0,7,7,-320.0,7,197.5,197.5
7278,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE,2017-33-0.0,703.0,V,28.0,22.0,21.0,22.0,93.0,5.5,5.5,-220.0,0.5,2017-33-0.0,704.0,H,22.0,19.0,37.0,21.0,99.0,210.5,212,190.0,102.5,5.5,210.5
7279,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE,2017-10-27.0,501.0,V,17.0,23.0,28.0,27.0,95.0,197.5,198.5,160.0,97,2017-10-27.0,502.0,H,26.0,20.0,25.0,26.0,97.0,2,4,-180.0,pk,197.5,197.5


In [1045]:
#dropping columns we will definitely not need in our model
combined_better = merged_odds_logs.drop([ 'Date', 'Date_2', 'Rot', 'VH', '2H', 'Rot_2', 'VH_2', '2H_2'], axis =1)


In [1046]:
combined_better['totals'] = combined_better['totals'].astype(float)

In [1047]:
# kappa = .2 * std('nba_actualtotal') creating 3 different kappas to test the model on
combined_better['kappa_1'] = np.std(combined_better['Total_pts']) * .2
combined_better['kappa_2'] = np.std(combined_better['Total_pts']) * .3
combined_better['kappa_3'] = np.std(combined_better['Total_pts']) * .4

In [1048]:
intial_features = combined_better.drop(['1st', '2nd', '3rd', '4th', 'ML', '1st_2', '2nd_2', '3rd_2', '4th_2', 'Final', 'Final_2', 'ML_2'], axis = 1)

In [1049]:
# creating the thresholds for each kappa that we created
intial_features['k_1_under'] = np.rint(intial_features['totals'] - intial_features['kappa_1'])
intial_features['k_1_over'] =  np.rint(intial_features['totals'] + intial_features['kappa_1'])
intial_features['k_2_under'] =  np.rint(intial_features['totals'] - intial_features['kappa_2'])
intial_features['k_2_over'] =  np.rint(intial_features['totals'] + intial_features['kappa_2'])
intial_features['k_3_under'] =  np.rint(intial_features['totals'] - intial_features['kappa_3'])
intial_features['k_3_over'] = np.rint(intial_features['totals'] + intial_features['kappa_3'])

In [1050]:
intial_features

,Team,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Team_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Total_pts,home,visitor,Open,Close,Open_2,Close_2,over_under,totals,kappa_1,kappa_2,kappa_3,k_1_under,k_1_over,k_2_under,k_2_over,k_3_under,k_3_over
0,LAC,240.00,113.86,41.71,88.14,47.36,14.57,36.43,39.87,15.86,18.43,83.54,9.43,32.43,41.86,24.14,8.57,3.00,11.86,20.14,2.86,ATL,240.00,113.00,40.29,85.86,47.04,14.43,36.57,39.21,18.00,21.57,85.39,10.57,36.00,46.57,24.57,7.43,6.57,12.86,20.57,11.00,229,LAC,ATL,226,225,6.5,6.5,226,226.0,4.13165,6.197475,8.263299,222.0,230.0,220.0,232.0,218.0,234.0
1,IND,243.57,113.14,42.29,89.43,47.46,13.71,36.86,37.43,14.86,18.71,78.29,9.43,32.43,41.86,28.00,8.71,6.29,15.29,20.29,0.29,MIL,243.57,118.57,43.57,92.43,47.20,12.86,37.43,34.47,18.57,22.29,84.79,9.86,39.43,49.29,25.71,7.86,4.29,15.43,16.57,4.43,253,MIL,IND,232,232.5,10,6.5,232,232.0,4.13165,6.197475,8.263299,228.0,236.0,226.0,238.0,224.0,240.0
2,HOU,240.00,103.43,38.57,91.00,42.74,11.29,38.43,29.47,15.00,22.43,69.23,10.29,31.43,41.71,21.57,7.57,4.71,13.57,18.71,-14.71,TOR,240.00,110.43,38.86,91.00,42.81,16.14,44.00,36.79,16.57,20.86,79.86,9.00,28.29,37.29,26.29,8.29,4.43,12.86,24.00,-8.43,216,HOU,TOR,8.5,8.5,223.5,225.5,8.5,223.5,4.13165,6.197475,8.263299,219.0,228.0,217.0,230.0,215.0,232.0
3,BOS,240.00,114.57,43.43,88.86,49.17,14.14,37.00,38.17,13.57,18.86,76.07,11.43,32.57,44.00,24.57,8.00,5.86,13.29,20.14,1.71,MEM,240.00,107.57,40.14,89.86,44.56,8.86,28.00,32.03,18.43,24.14,78.74,10.43,37.00,47.43,24.43,8.14,4.14,13.43,19.29,-2.29,258,MEM,BOS,221.5,218.5,1,4.5,221.5,221.5,4.13165,6.197475,8.263299,217.0,226.0,215.0,228.0,213.0,230.0
4,UTA,243.57,119.14,40.86,89.29,45.84,16.71,42.71,38.99,20.71,26.29,78.73,11.00,35.57,46.57,22.71,7.71,5.43,15.14,20.00,-1.14,CHI,243.57,108.86,41.29,89.57,46.14,13.57,34.71,40.04,12.71,15.71,83.87,11.14,36.71,47.86,27.43,6.00,4.71,16.43,19.43,2.00,215,CHI,UTA,8.5,9.5,231,227.5,8.5,231.0,4.13165,6.197475,8.263299,227.0,235.0,225.0,237.0,223.0,239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7276,LAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223,LAL,MIN,202.5,202.5,3.5,2.5,202.5,202.5,4.13165,6.197475,8.263299,198.0,207.0,196.0,209.0,194.0,211.0
7277,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,ATL,DET,197.5,196.5,7,7,197.5,197.5,4.13165,6.197475,8.263299,193.0,202.0,191.0,204.0,189.0,206.0
7278,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE,5.5,5.5,210.5,212,5.5,210.5,4.13165,6.197475,8.263299,206.0,215.0,204.0,217.0,202.0,219.0
7279,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,CHI,CLE,197.5,198.5,2,4,197.5,197.5,4.13165,6.197475,8.263299,193.0,202.0,191.0,204.0,189.0,206.0


In [1051]:
intial_features.to_csv('data/ready_for_ml_7.csv')

In [153]:
# label_encoder = LabelEncoder()
# team_encoded = label_encoder.fit_transform(intial_features['Team'] )
# for i in team_encoded:
#     print(i)
    
# team_encoded.classes_

In [131]:
one_hot.fit(teams)

OneHotEncoder(handle_unknown='ignore')